In [1]:
import requests
# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# nltk.download('stopwords')
# nltk.download('punkt')

import calendar
from datetime import datetime
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import random
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import datetime, timedelta

In [3]:
domain_ratings = pd.read_csv('domain_ratings.csv')
print(domain_ratings)

                         domain       pc1       afm  afm_bias   afm_min  \
0               theguardian.com  0.749592  0.777600  0.692600  0.692600   
1                     nhk.or.jp  0.867081  0.824299  0.776429  0.774133   
2                   nytimes.com  0.858776  0.813700  0.745400  0.745400   
3                   reuters.com  1.000000  0.962600  0.950100  0.950100   
4                       cnn.com  0.658031  0.785700  0.716200  0.716200   
...                         ...       ...       ...       ...       ...   
11515        thejewishvoice.com  0.331731  0.477951  0.430716  0.432535   
11516  thelibertyrevolution.com  0.351909  0.492022  0.434612  0.433924   
11517     usacrime.com/hoodsite  0.463725  0.521206  0.465516  0.464217   
11518         wndnewscenter.org  0.292802  0.347520  0.293491  0.316920   
11519              yournews.com  0.218392  0.157017  0.161997  0.155486   

       afm_rely        fc      mbfc  mbfc_bias  mbfc_fact  mbfc_min  \
0      0.862600  0.771860  0

In [ ]:

#%% 
# Quantitative Assessment

#Descriptive Statistics

# Summary statistics for pc1 values
print(domain_ratings['pc1'].describe())

# Count of domains above and below the threshold
print("Count of domains with pc1 >= 0.5:", domain_ratings[domain_ratings['pc1'] >= 0.5].shape[0])
print("Count of domains with pc1 < 0.5:", domain_ratings[domain_ratings['pc1'] < 0.5].shape[0])

#%%
#Visual Distribution


sns.histplot(domain_ratings['pc1'], kde=True)
plt.axvline(0.5, color='red', linestyle='--')  # Threshold line
plt.show()

#%%
# Qualitative Assessment 
# Sample Domains Above Threshold:
sample_high_quality = domain_ratings[domain_ratings['pc1'] >= 0.5].sample(n=10)
print(sample_high_quality)



#%%

# Keep only the domains with a rating of 0.3 or higher
high_quality_domains = domain_ratings[(domain_ratings['pc1'] >= 0.3)]['domain'].tolist()



# print(high_quality_domains)

# In[17]:

# API key from News API
api_key = '8558ff06892d4e99bab50611cbd38334'

# Endpoint URL
url = 'https://newsapi.org/v2/everything'

# Get today's date
today = datetime.now()

# Set the start of the month
start_of_month = datetime(today.year, today.month, 1)

# Convert dates to string format for the API request
today_str = today.strftime('%Y-%m-%d')
start_of_month_str = start_of_month.strftime('%Y-%m-%d')

# Parameters for the request
params = {
    'q': 'climate change OR global warming OR greenhouse gases OR climate crisis OR sustainability OR renewable energy',
    'from': start_of_month_str,
    'to': today_str,
    'sortBy': 'popularity',
    'apiKey': api_key
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    articles = data['articles']
else:
    print(f"Request failed with status code {response.status_code}")
#%%
    
import pandas as pd

# Convert the list of dictionaries into a DataFrame
df_articles = pd.DataFrame(articles)

# Display the DataFrame
print(df_articles)

#%%

def ensure_dict_format(entry):
    """
    This function ensures that the entry is in dictionary format.
    If it's a string, it converts it to a dictionary.
    """
    if isinstance(entry, dict):
        return entry
    elif isinstance(entry, str):
        return {'id': None, 'name': entry}
    else:
        raise TypeError(f"Unexpected type {type(entry)} encountered in the source column")

# Apply the function to the source column
df_articles['source'] = df_articles['source'].apply(ensure_dict_format)

#%%




#%%
def generate_domain_representations(source_dict):
    name = source_dict['name']  # Extract the name from the source dictionary
    representations = set()
    
    
    # Convert to lowercase and remove spaces
    base_name = name.lower().replace(" ", "")
    
    # If it doesn't have a '.' we'll assume it's a proper name and not a domain
    if '.' not in base_name:
        representations.add(base_name + ".com")
        representations.add(base_name + ".org")
        representations.add(base_name + ".net")
        representations.add("www." + base_name + ".com")
    else:
        representations.add(base_name)
        
        # Remove www.
        if base_name.startswith("www."):
            representations.add(base_name[4:])
        
        # Change .com to .co.uk and vice versa
        if base_name.endswith(".com"):
            representations.add(base_name[:-4] + ".co.uk")
        elif base_name.endswith(".co.uk"):
            representations.add(base_name[:-6] + ".com")
    
    return representations

# Add a new column to df_articles for the generated domain representations
df_articles['generated_domains'] = df_articles['source'].apply(generate_domain_representations)

print(df_articles[['source', 'generated_domains']])




#%%

# 1. Extract domain from the URL and store in a new column
from urllib.parse import urlparse

def extract_domain(url):
    return urlparse(url).netloc

df_articles['domain'] = df_articles['url'].apply(extract_domain)

# 2. Go through each article's domain and check against the domains_to_check and high-quality domains list
domains_to_check = {
    'ABC News': ['abcnews.go.com'],
    'Bezosearthfund.org': ['bezosearthfund.org'],
    'Boing Boing': ['boingboing.net'],
    'Business Insider': ['businessinsider.com'],
    'Colorado Public Radio': ['cpr.org'],
    'Engadget': ['engadget.com'],
    'Hubspot.com': ['hubspot.com'],
    'HuffPost': ['huffpost.com', 'huffingtonpost.com'],
    'NPR': ['npr.org'],
    'Noahpinion.blog': ['noahpinion.blog'],
    'PBS': ['pbs.org'],
    'Politicopro.com': ['politicopro.com'],
    'ReadWrite': ['readwrite.com'],
    'Sanjoseinside.com': ['sanjoseinside.com'],
    'Slashdot.org': ['slashdot.org'],
    'Slaynews.com': ['slaynews.com'],
    'The Atlantic': ['theatlantic.com'],
    'The Guardian': ['theguardian.com', 'guardian.co.uk'],
    'The New Yorker': ['newyorker.com'],
    'The Next Web': ['thenextweb.com'],
    'The Verge': ['theverge.com'],
    'The Washington Post': ['washingtonpost.com'],
    'Time': ['time.com'],
    'VentureBeat': ['venturebeat.com'],
    'WOODTV.com': ['woodtv.com'],
    'Yahoo Entertainment': ['yahoo.com', 'entertainment.yahoo.com']
}

found_domains = []
not_found_domains = []

print(df_articles['source'].apply(type).value_counts())


for index, row in df_articles.iterrows():
    domain = row['domain']
    source_dict = row['source']
    possible_representations = domains_to_check.get(source_dict['name'], [])
    
    # Extend possible representations using the generator function if the name doesn't exist in the domains_to_check
    if not possible_representations:
        possible_representations = generate_domain_representations(source_dict)  # <-- Pass the entire source dictionary here
    
    
    if any(rep in high_quality_domains for rep in possible_representations):
        found_domains.append(domain)
    else:
        not_found_domains.append(domain)
        df_articles.drop(index, inplace=True)  # Remove the row from the dataframe

print(f"Found Domains: {found_domains}")
print(f"Not Found Domains: {not_found_domains}")

#%%
import spacy
import spacy.cli
spacy.cli.download("en_core_web_sm")

nlp = spacy.load("en_core_web_sm")

def has_broad_keyword(content):
    # List of broad keywords related to climate change
    broad_keywords = [
        "climate", "carbon", "greenhouse", "global warming", "sustainability", 
        "renewable", "deforest", "emission", "fossil fuel", "solar", 
        "wind energy", "recycle", "conserve", "environment", "pollute", 
        "eco-friendly", "clean energy", "EV", "electric vehicle", "zero-emission",
        "carbon-neutral", "biofuel", "geothermal", "hydropower", "tidal energy", 
        "ozone", "melt glacier", "sea-level rise", "greenwash"
    ]
    
    # Process content with spaCy to get lemmas
    doc = nlp(content.lower())
    lemmatized_content = [token.lemma_ for token in doc]
    
    # Check if any of the lemmatized words match our keywords
    for keyword in broad_keywords:
        if keyword in lemmatized_content:
            return True
            
    return False
#%%
broad_keyword_articles = []

for index, row in df_articles.iterrows():
    content = row['content']
    if has_broad_keyword(content):
        broad_keyword_articles.append(row)

df_broad_keyword_articles = pd.DataFrame(broad_keyword_articles)


In [6]:
domain_ratings["pc1"].min()

0.0

In [7]:
domain_ratings["pc1"].max()

1.0

In [8]:
domain_ratings[domain_ratings["pc1"]<0.5]

,domain,pc1,afm,afm_bias,afm_min,afm_rely,fc,mbfc,mbfc_bias,mbfc_fact,mbfc_min,lewandowsky_acc,lewandowsky_trans,lewandowsky_rely,lewandowsky_mean,lewandowsky_min,misinfome_bin
13,dailymail.co.uk,0.383861,0.754900,0.938500,0.571200,0.571200,0.483500,0.500000,0.666700,0.333300,0.333300,0.000000,0.000000,0.000000,0.000000,0.000000,0
24,rumble.com,0.162506,0.344641,0.309389,0.317269,0.353323,0.048423,0.275896,0.507986,0.281422,0.287308,0.000000,0.000000,0.000000,0.000000,0.000000,0
29,rawstory.com,0.461915,0.525800,0.447400,0.447400,0.604300,0.098900,0.583300,0.666700,0.500000,0.500000,0.250000,0.500000,1.000000,0.375000,0.250000,1
30,indiatimes.com,0.388110,0.486057,0.463625,0.453684,0.512346,0.114944,0.666700,0.833300,0.500000,0.500000,0.250000,0.000000,0.000000,0.125000,0.000000,1
34,breitbart.com,0.297175,0.510900,0.463100,0.463100,0.558700,0.175800,0.500000,0.500000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11515,thejewishvoice.com,0.331731,0.477951,0.430716,0.432535,0.477314,0.067929,0.353661,0.522508,0.311092,0.256109,0.218946,0.499545,0.994115,0.389140,0.240574,0
11516,thelibertyrevolution.com,0.351909,0.492022,0.434612,0.433924,0.483467,0.044732,0.381495,0.510190,0.383213,0.356066,0.252006,0.499545,0.994557,0.390162,0.240639,0
11517,usacrime.com/hoodsite,0.463725,0.521206,0.465516,0.464217,0.558894,0.130961,0.663218,0.713930,0.616523,0.535566,0.245096,0.499545,0.989277,0.397674,0.249783,0
11518,wndnewscenter.org,0.292802,0.347520,0.293491,0.316920,0.356713,0.046789,0.373761,0.507986,0.328108,0.333284,0.249252,0.499545,0.992491,0.390136,0.240587,0


In [9]:
domain_ratings

,domain,pc1,afm,afm_bias,afm_min,afm_rely,fc,mbfc,mbfc_bias,mbfc_fact,mbfc_min,lewandowsky_acc,lewandowsky_trans,lewandowsky_rely,lewandowsky_mean,lewandowsky_min,misinfome_bin
0,theguardian.com,0.749592,0.777600,0.692600,0.692600,0.862600,0.771860,0.666700,0.833300,0.500000,0.500000,0.750000,1.000000,1.000000,0.875000,0.750000,1
1,nhk.or.jp,0.867081,0.824299,0.776429,0.774133,0.882758,0.637476,0.916700,1.000000,0.833300,0.833300,0.750000,1.000000,1.000000,0.875000,0.750000,1
2,nytimes.com,0.858776,0.813700,0.745400,0.745400,0.882000,1.000000,0.833300,0.833300,0.833300,0.833300,0.750000,1.000000,1.000000,0.875000,0.750000,1
3,reuters.com,1.000000,0.962600,0.950100,0.950100,0.975100,0.666580,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1
4,cnn.com,0.658031,0.785700,0.716200,0.716200,0.855300,0.923100,0.583300,0.666700,0.500000,0.500000,0.500000,0.500000,1.000000,0.500000,0.500000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11515,thejewishvoice.com,0.331731,0.477951,0.430716,0.432535,0.477314,0.067929,0.353661,0.522508,0.311092,0.256109,0.218946,0.499545,0.994115,0.389140,0.240574,0
11516,thelibertyrevolution.com,0.351909,0.492022,0.434612,0.433924,0.483467,0.044732,0.381495,0.510190,0.383213,0.356066,0.252006,0.499545,0.994557,0.390162,0.240639,0
11517,usacrime.com/hoodsite,0.463725,0.521206,0.465516,0.464217,0.558894,0.130961,0.663218,0.713930,0.616523,0.535566,0.245096,0.499545,0.989277,0.397674,0.249783,0
11518,wndnewscenter.org,0.292802,0.347520,0.293491,0.316920,0.356713,0.046789,0.373761,0.507986,0.328108,0.333284,0.249252,0.499545,0.992491,0.390136,0.240587,0
